# Import Libraries

In [3]:
import networkx as nx
import pandas as pd


import os, sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
from Backbones.high_salience_skeleton import high_salience_skeleton as hss
from Backbones.doubly_stochastic import read
from Backbones.marginal_likelihood_filter import MLF
from Backbones import disparity_filter as disf
from Backbones.noise_corrected import noise_corrected as nc
from Backbones.lans_backbone import lans


from Utils.code_utils import convert_label_to_integers, relabel_nodes


# Import the Netowrk

In [4]:
# initialize the network name and dataset path
network = 'airports'
path = '../Results/All Backbones/'
#path = '../Datasets/'

# read edge list from csv file
edge_list = pd.read_csv( path+ network + '.csv')
edge_list.weight = edge_list.weight.astype(int)

# read edge list from csv file for the doubly stochastic filter, noice corrected and the high salience skeleton
table, nnodes, nnedges = read(path + network + '.csv', "weight", sep=',', consider_self_loops=False, triangular_input = True, undirected=True) 
table.weight = table.weight.astype(int)  

# create graph from the edge list
G = nx.from_pandas_edgelist(edge_list, edge_attr='weight', create_using=nx.Graph())


# Treat Network

In [5]:
# to do
# later

# Global Threshold

In [6]:
# add alpha values to the original graph
for u,v, weight in G.edges(data='weight'):
    G[u][v]['global_score'] = weight

# Disparity Filter

In [7]:
# apply the disparity filter algorithm
backbone = disf.disparity_filter(G)

# add alpha values to the original graph
for u,v, alpha in backbone.edges(data='alpha'):
    G[u][v]['df_alpha'] = alpha

## The High Saleince Skeleton

In [8]:
# apply the high salience skeleton algorithm
# hss_backbone = hss(table, return_self_loops=False, undirected=True)
    
# # sort score values of the edges
# hss_backbone = hss_backbone.rename(columns={'score': 'threshold'})

# # add score values to the original graph
# for u, v in G.edges():
#     G[u][v]['hss_score'] = hss_backbone[((hss_backbone['source'] == u) & (hss_backbone['target'] == v)) | ((hss_backbone['source'] == v) & (hss_backbone['target'] == u))]['threshold'].iloc[-1]
   

# Marginal Likelihood Filter

In [9]:
# Apply the transformer to the edgelist
mlf = MLF(directed=False)
mlf_backbone = mlf.fit_transform(edge_list)

#sort score values of the edges
mlf_backbone = mlf_backbone.rename(columns={'significance': 'threshold'})

# add score values to the original graph
for u, v in G.edges():
    G[u][v]['mlf_score'] = mlf_backbone[((mlf_backbone['source'] == u) & (mlf_backbone['target'] == v)) | ((mlf_backbone['source'] == v) & (mlf_backbone['target'] == u))]['threshold'].iloc[-1]

# Noise Corrected Filter

In [10]:
#apply the noise corrected filter
#nc_backbone = nc(table, undirected = True, return_self_loops = False, calculate_p_value=True)
nc_backbone = nc(table, undirected = True, return_self_loops = False, calculate_p_value=True)
#nc_backbone['threshold'] = nc_backbone["score"] - (1 * nc_backbone["sdev_cij"])
nc_backbone['threshold'] = 1 - nc_backbone["score"]

# add score values to the original graph
for u, v in G.edges():
    G[u][v]['nc_alpha'] = nc_backbone[((nc_backbone['source'] == u) & (nc_backbone['target'] == v)) | ((nc_backbone['source'] == v) & (nc_backbone['target'] == u))]['threshold'].iloc[-1]

## The Polya Filter

In [11]:
#read edge list from csv file
pf_backbone = pd.read_csv('../Results/Backbones Results/PF/' + network + '.csv')

#sort score values of the edges
pf_backbone = pf_backbone.rename(columns={'p_values': 'threshold'})


# add score values to the original graph
for u, v in G.edges():
    G[u][v]['pf_alpha'] = pf_backbone[((pf_backbone['source'] == u) & (pf_backbone['target'] == v)) | ((pf_backbone['source'] == v) & (pf_backbone['target'] == u))]['threshold'].iloc[-1]

## Gloss

In [12]:
#read edge list from csv file
gloss_backbone = pd.read_csv('../Results/Backbones Results/GLOSS/' + network + '.csv', names=['source', 'target', 'threshold', 'weight'], sep=' ')

# #for lesmis
# labels = pd.read_csv('../Datasets/' + network + '_labels.csv', index_col='index')

# #relabel using saved labels
# labels = labels.T.to_dict('list')
# gloss_backbone['source'] = gloss_backbone['source'].replace(labels)
# gloss_backbone['target'] = gloss_backbone['target'].replace(labels)


# # add score values to the original graph
for u, v in G.edges():
    G[u][v]['gloss_alpha'] = gloss_backbone[((gloss_backbone['source'] == u) & (gloss_backbone['target'] == v)) | ((gloss_backbone['source'] == v) & (gloss_backbone['target'] == u))]['threshold'].iloc[-1]

# Lans Filter

In [13]:
g = lans(G)

for u, v in G.edges():
    G[u][v]['lans_alpha'] = g[u][v]['p-value']

# ECM Filter

In [14]:
# conda install -c conda-forge jaxlib
# conda install -c conda-forge jax
# pip install tabulate

from Backbones.ECM import ecm
import scipy.sparse

g, old_lables = convert_label_to_integers(G)

W = nx.adjacency_matrix(g, weight="weight")

model = ecm.ECM(W)
initial_guess = model.get_initial_guess(option=1)
solution = model.solve(initial_guess, verbose=True)

pval_M = model.get_pval_matrix(solution.x, W)
lower_pval_M = scipy.sparse.tril(pval_M).toarray()

for (i,j) in zip(*lower_pval_M.nonzero()):
    p = lower_pval_M[i,j]
    g[i][j]['ecm_alpha'] = p 
    
g = relabel_nodes(g, old_lables)

for u, v in G.edges():
    G[u][v]['ecm_alpha'] = g[u][v]['ecm_alpha']
    

DEBUG:absl:Initializing backend 'interpreter'
DEBUG:absl:Backend 'interpreter' initialized
DEBUG:absl:Initializing backend 'cpu'
DEBUG:absl:Backend 'cpu' initialized
DEBUG:absl:Initializing backend 'tpu_driver'
INFO:absl:Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
DEBUG:absl:Initializing backend 'gpu'
INFO:absl:Unable to initialize backend 'gpu': NOT_FOUND: Could not find registered platform with name: "cuda". Available platform names are: Host Interpreter
DEBUG:absl:Initializing backend 'tpu'
INFO:absl:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
DEBUG:absl:Finished tracing + transforming transform_parameters_inv for jit in 0.0030879974365234375 sec
DEBUG:absl:Compiling transform_parameters_inv (140668530470496 for args (ShapedArray(float32[5468]),).
DEBUG:absl:Finished XLA compilation of transform_parameters_inv in 0.24920392036437988 sec
DEBUG:absl:Finished tracing + transforming neg_lo

Took 79.443359375 seconds
Relative error for expected degree/strength sequence: 

Percentile      Relative error
------------  ----------------
Min                0
25th               1.16654e-06
Median             2.92585e-06
75th               7.76337e-06
Max                0.000658751

Residual error: 64.61173248291016


# Export to csv

In [15]:
nx.to_pandas_edgelist(G).to_csv('../Results/All Backbones/'+network+'.csv', index=False)

# Modularity Backbone

In [14]:
from Backbones import modularity_backbone as modv

# k_nodes_to_remove = round(len(G.nodes())*0)
# backbone, Q1, Q2, Q31, Q32 = modv.modularity_backbone(G, k_nodes_to_remove)
# df_modularity = pd.DataFrame(backbone.nodes(data='modularity'), columns=['id', 'modularity']).sort_values(by='modularity', ascending=False)
# df_modularity.to_csv('../Results/All Backbones/modularit-'+network+'.csv', index=False)